In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

In [2]:
# Display educational header
display(HTML("""
<h2>Single-Molecule EDC Binding Events: Interactive Data Viewer</h2>
<p><strong>Objective:</strong> Observe the difference between background noise and molecular binding events.</p>
<ul>
    <li><strong>Background Data (18:05-18:21):</strong> Carbon nanotube in buffer without EDC molecules</li>
    <li><strong>EDC Data (18:26-18:59):</strong> EDC molecules present, showing binding/unbinding events</li>
</ul>
"""))

# Load decimated datasets
print("Loading datasets...")
background_df = pd.read_csv('background_data_decimated.csv')
edc_df = pd.read_csv('edc_full_data_decimated.csv')

print(f"✓ Background: {len(background_df):,} samples ({background_df['time_s'].iloc[-1]/60:.1f} min)")
print(f"✓ EDC: {len(edc_df):,} samples ({edc_df['time_s'].iloc[-1]/60:.1f} min)")

# Get max times for each dataset
bg_max_time = background_df['time_s'].iloc[-1]
edc_max_time = edc_df['time_s'].iloc[-1]

Loading datasets...
✓ Background: 4,727,369 samples (15.8 min)
✓ EDC: 10,153,547 samples (33.8 min)


In [3]:
def plot_single_dataset(dataset_name, start_time, window_duration):
    """Plot a time window from one dataset"""
    
    # Select dataset and styling
    if dataset_name == 'Background (No EDC)':
        df = background_df
        color = 'navy'
        max_time = bg_max_time
        title_prefix = 'Background'
    else:
        df = edc_df
        color = 'darkred'
        max_time = edc_max_time
        title_prefix = 'EDC Present'
    
    # Validate time range
    if start_time >= max_time:
        print(f"⚠ Start time {start_time}s exceeds dataset duration ({max_time:.1f}s)")
        return
    
    # Extract window
    end_time = min(start_time + window_duration, max_time)
    mask = (df['time_s'] >= start_time) & (df['time_s'] <= end_time)
    window = df.loc[mask]
    
    if len(window) == 0:
        print(f"⚠ No data in selected window")
        return
    
    # Create plot
    fig, ax = plt.subplots(figsize=(15, 5))
    
    ax.plot(window['time_s'].values, window['Isd'].values, 
            linewidth=1, color=color, alpha=0.8, label='Current')
    
    # Add mean line
    mean_current = window['Isd'].mean()
    ax.axhline(mean_current, color='blue', linestyle='--', 
               linewidth=2, alpha=0.5, label=f'Mean: {mean_current:.6f} A')
    
    ax.set_xlabel('Time (seconds)', fontsize=12)
    ax.set_ylabel('Current (A)', fontsize=12)
    ax.set_title(f'{title_prefix}: {start_time:.1f}s to {end_time:.1f}s', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(alpha=0.3)
    
    # Add statistics box
    stats_text = f'Duration: {end_time - start_time:.1f} s\n'
    stats_text += f'Samples: {len(window):,}\n'
    stats_text += f'Mean: {window["Isd"].mean():.6f} A\n'
    stats_text += f'Std Dev: {window["Isd"].std():.6f} A\n'
    stats_text += f'Range: [{window["Isd"].min():.6f}, {window["Isd"].max():.6f}]'
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
            verticalalignment='top', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Create interactive widgets
print("\n" + "="*60)
print("INTERACTIVE VIEWER: Single Dataset")
print("="*60)

dataset_selector = widgets.Dropdown(
    options=['Background (No EDC)', 'EDC Present (Fluctuations)'],
    value='Background (No EDC)',
    description='Dataset:',
    style={'description_width': 'initial'}
)

start_time_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=300,
    step=1,
    description='Start Time (s):',
    continuous_update=False,
    readout_format='.1f',
    style={'description_width': 'initial'}
)

duration_selector = widgets.Dropdown(
    options=[('1 second', 1), ('5 seconds', 5), ('10 seconds', 10), 
             ('30 seconds', 30), ('1 minute', 60), ('5 minutes', 300)],
    value=10,
    description='Window:',
    style={'description_width': 'initial'}
)

# Create interactive plot
single_viewer = widgets.interactive(
    plot_single_dataset,
    dataset_name=dataset_selector,
    start_time=start_time_slider,
    window_duration=duration_selector
)

display(single_viewer)


INTERACTIVE VIEWER: Single Dataset


interactive(children=(Dropdown(description='Dataset:', options=('Background (No EDC)', 'EDC Present (Fluctuati…

In [4]:
def compare_datasets(start_time, window_duration):
    """Compare background and EDC data side-by-side at the same time point"""
    
    fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    
    end_time = start_time + window_duration
    
    # Background data
    bg_mask = (background_df['time_s'] >= start_time) & (background_df['time_s'] <= end_time)
    bg_window = background_df.loc[bg_mask]
    
    if len(bg_window) > 0:
        axes[0].plot(bg_window['time_s'].values, bg_window['Isd'].values,
                     linewidth=1, color='navy', alpha=0.8)
        axes[0].axhline(bg_window['Isd'].mean(), color='blue', linestyle='--', 
                       linewidth=2, alpha=0.5, label=f'Mean: {bg_window["Isd"].mean():.6f} A')
        axes[0].set_ylabel('Current (A)', fontsize=12)
        axes[0].set_title('Background (No EDC): Pure 1/f Noise', fontsize=13, fontweight='bold')
        axes[0].legend(loc='upper right')
        axes[0].grid(alpha=0.3)
        
        # Stats for background
        bg_stats = f'Std: {bg_window["Isd"].std():.6f} A'
        axes[0].text(0.02, 0.02, bg_stats, transform=axes[0].transAxes,
                    fontsize=10, bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
    
    # EDC data
    edc_mask = (edc_df['time_s'] >= start_time) & (edc_df['time_s'] <= end_time)
    edc_window = edc_df.loc[edc_mask]
    
    if len(edc_window) > 0:
        axes[1].plot(edc_window['time_s'].values, edc_window['Isd'].values,
                     linewidth=1, color='darkred', alpha=0.8)
        axes[1].axhline(edc_window['Isd'].mean(), color='blue', linestyle='--', 
                       linewidth=2, alpha=0.5, label=f'Mean: {edc_window["Isd"].mean():.6f} A')
        axes[1].set_xlabel('Time (seconds)', fontsize=12)
        axes[1].set_ylabel('Current (A)', fontsize=12)
        axes[1].set_title('EDC Present: Look for Two-Level Switching Events', fontsize=13, fontweight='bold')
        axes[1].legend(loc='upper right')
        axes[1].grid(alpha=0.3)
        
        # Stats for EDC
        edc_stats = f'Std: {edc_window["Isd"].std():.6f} A'
        axes[1].text(0.02, 0.02, edc_stats, transform=axes[1].transAxes,
                    fontsize=10, bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))
    
    fig.suptitle(f'Comparison: {start_time:.1f}s to {end_time:.1f}s', 
                 fontsize=15, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

print("\n" + "="*60)
print("INTERACTIVE VIEWER: Side-by-Side Comparison")
print("="*60)

comparison_viewer = widgets.interactive(
    compare_datasets,
    start_time=widgets.FloatSlider(
        value=0, min=0, max=min(bg_max_time, edc_max_time) - 60, step=5,
        description='Start Time (s):', continuous_update=False,
        readout_format='.1f', style={'description_width': 'initial'}
    ),
    window_duration=widgets.Dropdown(
        options=[('5 seconds', 5), ('10 seconds', 10), ('30 seconds', 30), 
                 ('1 minute', 60), ('2 minutes', 120)],
        value=10, description='Window:',
        style={'description_width': 'initial'}
    )
)

display(comparison_viewer)


INTERACTIVE VIEWER: Side-by-Side Comparison


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Start Time (s):', max=885.4…

In [5]:
def plot_multiple_windows(dataset_name, n_windows, window_duration):
    """Plot multiple evenly-spaced windows in a grid"""
    
    # Select dataset
    if dataset_name == 'Background (No EDC)':
        df = background_df
        color = 'navy'
        max_time = bg_max_time
        title = 'Background'
    else:
        df = edc_df
        color = 'darkred'
        max_time = edc_max_time
        title = 'EDC Present'
    
    # Calculate window positions
    available_duration = max_time - window_duration
    start_times = np.linspace(0, available_duration, n_windows)
    
    # Create grid
    n_cols = 2
    n_rows = (n_windows + 1) // 2
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
    axes = axes.flatten() if n_windows > 1 else [axes]
    
    for idx, start_time in enumerate(start_times):
        end_time = start_time + window_duration
        mask = (df['time_s'] >= start_time) & (df['time_s'] <= end_time)
        window = df.loc[mask]
        
        if len(window) > 0:
            axes[idx].plot(window['time_s'].values, window['Isd'].values,
                          linewidth=1, color=color, alpha=0.8)
            axes[idx].set_xlabel('Time (s)', fontsize=10)
            axes[idx].set_ylabel('Current (A)', fontsize=10)
            axes[idx].set_title(f't = {start_time/60:.1f} min', fontsize=11)
            axes[idx].grid(alpha=0.3)
    
    # Hide unused subplots
    for idx in range(n_windows, len(axes)):
        axes[idx].axis('off')
    
    fig.suptitle(f'{title}: {n_windows} samples across dataset', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("\n" + "="*60)
print("INTERACTIVE VIEWER: Multiple Windows Grid")
print("="*60)

grid_viewer = widgets.interactive(
    plot_multiple_windows,
    dataset_name=widgets.Dropdown(
        options=['Background (No EDC)', 'EDC Present (Fluctuations)'],
        value='EDC Present (Fluctuations)',
        description='Dataset:', style={'description_width': 'initial'}
    ),
    n_windows=widgets.IntSlider(
        value=4, min=2, max=8, step=2,
        description='# Windows:', style={'description_width': 'initial'}
    ),
    window_duration=widgets.Dropdown(
        options=[('5 seconds', 5), ('10 seconds', 10), ('30 seconds', 30)],
        value=10, description='Window Size:',
        style={'description_width': 'initial'}
    )
)

display(grid_viewer)


INTERACTIVE VIEWER: Multiple Windows Grid


interactive(children=(Dropdown(description='Dataset:', index=1, options=('Background (No EDC)', 'EDC Present (…